# Python Notes: Simple APIs
<hr>

API: Application Program Interfaces  
  
APIs allows to programs to communicate with one another, through inputs and outputs  
> Pandas is an example of usage of an API  
>> Pandas has components that are not written in Python, meaning there is a separate program that handles the requests for operations with the Pandas objects and operations like dataframes and requests for the first five rows through the head() method  
  
REST API: REpresentational State Transfer API  

REST APIs are used to interact with web services, such as applications that uses the internet  
> REST APIs use a different set of jargon to describe its components:  
> it consists of a client (my program) and the web service / resource (the other program) communicating through requests (inputs) and responses (outputs) through the internet 
>> functions similarly but with different names  
>>> REST APIs have rules regarding communication, requests, and responses  
  
REST APIs over HTTP  
HTTP methods are ways to transmit data over the internet  
> the request is communicated through an http message, which usually contains a JSON (JavaScript Object Notation) file, which has the instructions for the service to perform  
> the request is sent through the internet  
> the web service performs the operation and sends an http message containing a JSON file to the client  


PyCoinGecko is a wrapper (which is a language specific package that simplifies complex API calls needed to perform a common task with an associated API) that can be used to collect data about cryptocurrencies by using the CoinGeck API  
  
**!pip install pycoingecko**  
**from pycoingecko import CoinGeckoAPI**  
**_myObject_ = CoinGeckoAPI()**  
**_myDataVar_ = _myObject_.get_coin_market_chart_by_id(id = "_bitcoin_", vs_currency = "_usd_", days = _30_)**  
  
Returns a dictionary of information about the requested cryptocurrency  


<hr>

API Keys and Endpoints are needed to access the APIs  
The API Key provides a way to access the API by identifying users with it as an authorized user of the API  
> The first call to the API usually involves a call with the API key, which will grant access to the API  
  

The API endpoint is the location of the service, used to find the API in the internet  

Using an API usually involves importing the package that has the object which communicates with the API, the key, and the endpoint  
> the object which interacts with the API is created, with the key and endpoint as attributes  
> methods are then used to send files/requests to the API to the web service 
>> when methods are called, the returned value is usually the response from the web service  
>>> the response are usually in dictionary form since the response is sent back as a JSON, which are dictionaries  
> the response then needs to be processed to focus on the relevant information returned  
>> sometimes a method has to be applied on the response to access the data needed  

##### Use of API wrappers are highly dependent on the wrapper itself, so template codes will not be available in the Markdown cells since it may vary too much  
  
_A sample of acquiring data from an API will be below as reference to the generalized steps described in the cell above; the cells below are cells provided in the IBM Data Science Course to use as reference with the Markdown cells as personal comments and annotations explaining what is happening_

<hr>

The cell below installs the necessary packages so that it can be imported for use

In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance
!pip install --upgrade nbformat

     ---------------------------------------- 62.8/62.8 kB 3.5 MB/s eta 0:00:00
     -------------------------------------- 140.6/140.6 kB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 161.1/161.1 kB 9.4 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


     --------------------------------------- 15.3/15.3 MB 17.2 MB/s eta 0:00:00
     ---------------------------------------- 71.6/71.6 kB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 27.1 MB/s eta 0:00:00
     ---------------------------------------- 55.3/55.3 kB 2.8 MB/s eta 0:00:00
     ------------------------------------- 965.4/965.4 kB 12.2 MB/s eta 0:00:00
     -------------------------------------- 163.6/163.6 kB 9.6 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 26.1 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


     ---------------------------------------- 77.1/77.1 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 90.2/90.2 kB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 58.8/58.8 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 62.8/62.8 kB 3.3 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


<hr>
  
Imports the necessary libraries

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

<hr>  
  
1. Creates a variable _cg_ which references to an initialized CoinGeckoAPI object  
2. Creates a variable _bitcoin_data_ which stores the response from the request made by using the method *.get_coin_market_chart_by_id()* with the given parameters  

In [3]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

Returns a dictionary of the bitcoin data requested

In [4]:
type(bitcoin_data )

dict

1. Variable _bitcoin_price_data_ is created to return only the values related to the dictionary key "prices"  
2. The first five values are then returned

In [5]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1667081010952, 20868.336606180965],
 [1667084481493, 20731.806711867357],
 [1667088224784, 20800.70708772471],
 [1667091801871, 20728.173555634443],
 [1667095351240, 20747.464833701066]]

The values from the "prices" dictionary key is then casted into a dataframe "data", making the first element of each nested list to have the "TimeStamp" column and the second element with the "Price" column

In [6]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

A "date" column is created  
> it is based on the "TimeStamp" column, applying transformations so that it is more readable  
>> converts the timestamps provided into a datetime format  

In [7]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))


* Groups by the newly created "date" column and assigns it to the variable "candlestick_date" 
> aggregates the following data about the price: "min, max, first, last"  
>> each group will have the aggregated data for that date  

In [8]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

* Plots the data collected as a candlestick chart

In [9]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()